# WORKFLOW Data Preprocessing Summary

1. LOAD original dataset:
2. DATA CLEANING:

Tools:
* Panda -> load data, manipulate data, create new features

Output:
data -> Cleaned_HouseListings-Top45Cities.csv

1. LOAD original dataset locally:
* Download dataset on work directory and install pandas library.

In [185]:
import pandas as pd
# Set the file path (ensure the CSV file is in the correct folder)
file_path = "HouseListings-Top45Cities-10292023-kaggle.csv"

# Load the dataset
data = pd.read_csv(file_path, encoding='latin-1')

2.  DATA CLEAN: 
* Preview data to check Number of NaN values: 0 and Number of duplicated rows

In [186]:
# Preview the first few rows
print("Dataset preview:")
print(data.head())

Dataset preview:
      City      Price                  Address  Number_Beds  Number_Baths  \
0  Toronto   779900.0    #318 -20 SOUTHPORT ST            3             2   
1  Toronto   799999.0    #818 -60 SOUTHPORT ST            3             1   
2  Toronto   799900.0  #714 -859 THE QUEENSWAY            2             2   
3  Toronto  1200000.0         275 MORTIMER AVE            4             2   
4  Toronto   668800.0    #420 -388 RICHMOND ST            1             1   

  Province  Population  Latitude  Longitude  Median_Family_Income  
0  Ontario     5647656   43.7417   -79.3733               97000.0  
1  Ontario     5647656   43.7417   -79.3733               97000.0  
2  Ontario     5647656   43.7417   -79.3733               97000.0  
3  Ontario     5647656   43.7417   -79.3733               97000.0  
4  Ontario     5647656   43.7417   -79.3733               97000.0  


In [187]:
# Display data types, non-null counts, and duplicated rows
print("\nDataset info:")
print(data.info())

print("Number of NaN values:",data.isna().sum().sum())
print("Number of duplicated rows:",data.duplicated().sum())


Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35768 entries, 0 to 35767
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   City                  35768 non-null  object 
 1   Price                 35768 non-null  float64
 2   Address               35768 non-null  object 
 3   Number_Beds           35768 non-null  int64  
 4   Number_Baths          35768 non-null  int64  
 5   Province              35768 non-null  object 
 6   Population            35768 non-null  int64  
 7   Latitude              35768 non-null  float64
 8   Longitude             35768 non-null  float64
 9   Median_Family_Income  35768 non-null  float64
dtypes: float64(4), int64(3), object(3)
memory usage: 2.7+ MB
None
Number of NaN values: 0
Number of duplicated rows: 2516


2.  DATA CLEAN: 
* Drop duplicate rows

In [188]:
# Function to drop duplicate rows
def cleaning(Data):
    Data.drop_duplicates(inplace=True)
    Data.reset_index(drop=True, inplace=True)
    return Data

# Apply the cleaning function and display a random sample of 5 rows
cleaning(data).sample(5)

,City,Price,Address,Number_Beds,Number_Baths,Province,Population,Latitude,Longitude,Median_Family_Income
17755,Barrie,649000.0,#A109 -241 SEA RAY AVE,2,1,Ontario,154676,44.3711,-79.6769,97000.0
27540,Maple Ridge,1161900.0,117-3420 QUEENSTON AVENUE,4,3,British Columbia,90990,49.2167,-122.6000,105000.0
25654,White Rock,5988000.0,13585 28 AVENUE,7,8,British Columbia,109167,49.0250,-122.8028,73000.0
18019,Barrie,1599000.0,114 BEARBERRY RD,5,6,Ontario,154676,44.3711,-79.6769,97000.0
9178,Kitchener,1149900.0,19 SEATON CRES,3,2,Ontario,522888,43.4186,-80.4728,92000.0


2.  DATA CLEAN: 
* Improve consistency on province name using abbreviations for each province:
- `ON` for Ontario  
- `BC` for British Columbia  
- `QC` for Quebec  
- `AB` for Alberta

In [189]:
# Get province names and define standardized abbreviations for provinces
dataframe_province_name = data['Province'].unique()
Province_Abbrev_list = ["ON", "BC", "AB", "MB", "SK", "NL", "NB", "QC", "NS"]

In [190]:
# Create a dictionary from province names to abbreviations and replace in the dataset
dict_province_name = dict(zip(dataframe_province_name,Province_Abbrev_list))

for values, keys in dict_province_name.items():
   data.replace({"Province": {values: keys}} ,inplace = True)

In [191]:
# Verify that province names were successfully converted to abbreviations
data["Province"].unique()

array(['ON', 'BC', 'AB', 'MB', 'SK', 'NL', 'NB', 'QC', 'NS'], dtype=object)

In [192]:
# Check the number of listings per city in Ontario
data[data["Province"] == "ON"].City.value_counts()

City
Caledon             1331
London              1311
Toronto             1274
Hamilton            1266
St. Catharines      1259
Kitchener           1161
Ottawa              1093
Oshawa              1079
Barrie              1000
Windsor              718
Brantford            617
Kingston             471
Guelph               434
Peterborough         305
Sudbury              202
Sault Ste. Marie     165
Thunder Bay          154
Saskatoon              4
Winnipeg               1
Regina                 1
Nanaimo                1
Name: count, dtype: int64

In [193]:
# Check the number of listings per city in Newfoundland and Labrador
data[data["Province"] == "NL"].City.value_counts()

City
St. John's    652
Nanaimo         3
Name: count, dtype: int64

2.  DATA CLEAN: 
* Correction about some cities were mistakenly assigned to the wrong provinces.
- `Winnipeg` was under Ontario.
- `Regina` and `Saskatoon` were under Ontario
- `Nanaimo` was under Ontario.
- `Nanaimo` was under NL.

In [194]:
# Correct mislabeled cities that were wrongly assigned to Ontario
ontario = data[data.loc[:,"Province"]=="ON"]
index1 = ontario[ontario.City == "Winnipeg"].index
index2 = ontario[ontario.City == "Regina"].index
index3 = ontario[ontario.City == "Saskatoon"].index
index4 = ontario[ontario.City == "Nanaimo"].index

# Reassign correct provinces for those cities
data.loc[index1,"Province"] = "MB"
data.loc[index2,"Province"] = "SK"
data.loc[index3,"Province"] = "SK"
data.loc[index4,"Province"] = "BC"

# Correct mislabeled city in Newfoundland and Labrador
Newfoundland = data[data.loc[:,"Province"]=="NL"]
index5 = Newfoundland[Newfoundland.City == "Nanaimo"].index
data.loc[index5,"Province"] = "BC"

# Confirm that only valid cities remain in Ontario
data[data["Province"]=="ON"].City.value_counts()

City
Caledon             1331
London              1311
Toronto             1274
Hamilton            1266
St. Catharines      1259
Kitchener           1161
Ottawa              1093
Oshawa              1079
Barrie              1000
Windsor              718
Brantford            617
Kingston             471
Guelph               434
Peterborough         305
Sudbury              202
Sault Ste. Marie     165
Thunder Bay          154
Name: count, dtype: int64

In [195]:
# Confirm that only valid cities remain in NL
data[data["Province"]=="NL"].City.value_counts()

City
St. John's    652
Name: count, dtype: int64

2.  DATA CLEAN: 
* Review and correction of longitude value for Nova Scotia (NS), which was negative (`-63.1005`).

In [196]:
# Get the average latitude and longitude of each province
data.groupby("Province")[["Latitude","Longitude"]].mean()

,Latitude,Longitude
Province,,
AB,51.830823,-113.476187
BC,49.360071,-122.471522
MB,49.884400,-97.146400
NB,45.829295,-65.236060
NL,47.481700,-52.797100
NS,44.885700,63.100500
ON,43.797066,-79.740088
QC,46.014270,-72.219817
SK,51.269237,-105.614365


In [197]:
# Correct invalid longitude value for NS (+63.1005 should be -63.1005)
data.replace({"Longitude": {63.1005: -63.1005}} ,inplace = True)

# Confirm the corrected value
data.groupby("Province")[["Latitude","Longitude"]].mean()

,Latitude,Longitude
Province,,
AB,51.830823,-113.476187
BC,49.360071,-122.471522
MB,49.884400,-97.146400
NB,45.829295,-65.236060
NL,47.481700,-52.797100
NS,44.885700,-63.100500
ON,43.797066,-79.740088
QC,46.014270,-72.219817
SK,51.269237,-105.614365


2.  DATA CLEAN: 
* Some of houses with zero bathrooms will be removed from the dataset, as such entries are not realistic or useful for price prediction.

* The 'Address' column is removed as it provides no predictive value and is redundant with location data.

In [198]:
# Remove listings with 0 bedrooms
data = data[data['Number_Beds'] != 0 ].reset_index(drop = True)
#print("columnas:", data.shape)

# Drop the 'Address' column
data.drop('Address',axis = 1,inplace = True)
#print("columnas:", data.shape)

# Output:

In [199]:
# Save the cleaned dataset to a CSV file without the index
data.to_csv("Cleaned_HouseListings-Top45Cities.csv",index = False)